In [2]:
import os 
import pandas as pd
from collections import Counter
print("modules imported")

# establish paths
domain = "Rockies"
outputWorkspace = f"D:/ASOML/{domain}/modelOutputs/fromAlpine/"
model_runs = ["20250717_073937", "20250717_074016", "20250717_074248",
             "20250717_090321", "20250717_091138", "20250717_095915", "20250717_103354", "20250717_104948", 
              "20250717_105044", "20250717_115221", "20250717_115417", "20250717_115648", "20250717_135654", "20250718_230058", 
            "20250719_061602", "20250719_061740", "20250720_223902", "20250720_232408", "20250721_062531", "20250721_063239", "20250721_071615",
            "20250721_071631", "20250721_073243", "20250722_102255"]

groups = ["G1", "G2", "G3", "G4", "G5", "G6"]
top_k = 5  # Number of top models to keep per group
score_column = "RMSE"
group_column = "GroupNum"

# Initialize list to store all model data
all_model_data = []

# Architecture stats
architStats = f"{outputWorkspace}/{domain}_modelSummary_stats.csv"
if os.path.exists(architStats):
    arch_df = pd.read_csv(architStats)
    print(f"Loaded architecture stats with {len(arch_df)} records")
else:
    print(f"Architecture stats file not found: {architStats}")
    arch_df = pd.DataFrame()

# Process each model run
print(f"Processing {len(model_runs)} model runs...")
for i, run in enumerate(model_runs):
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(model_runs)} runs...")
    
    folder = f"{outputWorkspace}{run}/"
    errorCSV = f"{folder}/errorReview/{run}_error_summary_stats.csv"
    
    if os.path.exists(errorCSV):
        try:
            df = pd.read_csv(errorCSV)
            
            # Get architecture info for this run if available
            architecture = "Unknown"
            feature_num = "Unknown"
            final_activation = "Unknown"
            train_shape = "Unknown"
            valid_shape = "Unknown"
            
            if not arch_df.empty and 'ModelRun' in arch_df.columns:
                arch_match = arch_df[arch_df['ModelRun'] == run]
                if not arch_match.empty:
                    arch_row = arch_match.iloc[0]
                    architecture = arch_row.get('Architecture', 'Unknown')
                    feature_num = arch_row.get('FeatureNum', 'Unknown')
                    final_activation = arch_row.get('FinalActivation', 'Unknown')
                    train_shape = arch_row.get('X_TrainShape', 'Unknown')
                    valid_shape = arch_row.get('X_ValidShape', 'Unknown')
            
            for _, row in df.iterrows():
                group = row.get(group_column, 'Unknown')
                score = row.get(score_column, None)
                
                # Collect all available data for this model/group combination
                model_data = {
                    'ModelRun': run,
                    'Group': group,
                    'RMSE': score,
                    'Architecture': architecture,
                    'FeatureNum': feature_num,
                    'FinalActivation': final_activation,
                    'TrainShape': train_shape,
                    'ValidShape': valid_shape,
                    'Folder': folder,
                    'ErrorCSV': errorCSV
                }
                
                # Add all other columns from the error CSV
                for col in df.columns:
                    if col not in [group_column, score_column]:
                        model_data[f"Test_{col}"] = row.get(col, None)
                
                # Add training metrics from architecture CSV if available
                if not arch_df.empty and 'ModelRun' in arch_df.columns:
                    arch_match = arch_df[arch_df['ModelRun'] == run]
                    if not arch_match.empty:
                        arch_row = arch_match.iloc[0]
                        for col in arch_df.columns:
                            if col not in ['ModelRun', 'Architecture', 'FeatureNum', 'FinalActivation', 'X_TrainShape', 'X_ValidShape']:
                                model_data[f"Train_{col}"] = arch_row.get(col, None)
                
                all_model_data.append(model_data)
                    
        except Exception as e:
            print(f"Error reading {errorCSV}: {e}")
    else:
        # Still record that this run exists but has no test data
        all_model_data.append({
            'ModelRun': run,
            'Group': 'No_Test_Data',
            'RMSE': None,
            'Architecture': 'Unknown',
            'Status': 'Missing_Error_CSV',
            'Folder': folder,
            'ErrorCSV': errorCSV
        })

print(f"Collected data for {len(all_model_data)} model/group combinations")

# Create comprehensive DataFrame
comprehensive_df = pd.DataFrame(all_model_data)

# Save comprehensive CSV with ALL data
comprehensive_csv = f"{outputWorkspace}/comprehensive_model_results.csv"
comprehensive_df.to_csv(comprehensive_csv, index=False)
print(f"Comprehensive results saved to: {comprehensive_csv}")

# Now create the top K summary
print(f"\nCreating top {top_k} summary...")

# Filter for valid RMSE scores and groups
valid_data = comprehensive_df[
    (comprehensive_df['RMSE'].notna()) & 
    (comprehensive_df['Group'].isin(groups))
].copy()

print(f"Found {len(valid_data)} valid model/group combinations")

# Get top K for each group and track frequency
top_models_list = []
model_frequency = Counter()
model_details = {}

for group in groups:
    group_data = valid_data[valid_data['Group'] == group].copy()
    if not group_data.empty:
        # Sort by RMSE (ascending - lower is better) and take top K
        group_top = group_data.nsmallest(top_k, 'RMSE').copy()
        group_top['Rank_in_Group'] = range(1, len(group_top) + 1)
        top_models_list.append(group_top)
        
        # Count how many times each model appears in top K
        for _, row in group_top.iterrows():
            model_run = row['ModelRun']
            model_frequency[model_run] += 1
            
            # Store model details (architecture, etc.) for later use
            if model_run not in model_details:
                model_details[model_run] = {
                    'Architecture': row['Architecture'],
                    'FeatureNum': row['FeatureNum'],
                    'FinalActivation': row['FinalActivation'],
                    'Folder': row['Folder']
                }

# Combine all top models
if top_models_list:
    top_models_df = pd.concat(top_models_list, ignore_index=True)
    
    # Add overall rank across all groups
    top_models_df = top_models_df.sort_values('RMSE').reset_index(drop=True)
    top_models_df['Overall_Rank'] = range(1, len(top_models_df) + 1)
    
    # Reorder columns for better readability
    primary_cols = ['Overall_Rank', 'Group', 'Rank_in_Group', 'ModelRun', 'RMSE', 'Architecture', 
                   'FeatureNum', 'FinalActivation', 'TrainShape', 'ValidShape']
    other_cols = [col for col in top_models_df.columns if col not in primary_cols]
    top_models_df = top_models_df[primary_cols + other_cols]
    
    # Save top models summary
    top_models_csv = f"{outputWorkspace}/top_{top_k}_models_by_group.csv"
    top_models_df.to_csv(top_models_csv, index=False)
    print(f"Top {top_k} models summary saved to: {top_models_csv}")
    
    # =====================================================
    # ANALYZE MODEL FREQUENCY IN TOP 5
    # =====================================================
    print(f"\n{'='*80}")
    print(f"MODEL FREQUENCY ANALYSIS - TOP {top_k} ACROSS ALL GROUPS")
    print(f"{'='*80}")
    
    # Create frequency analysis DataFrame
    frequency_data = []
    for model_run, count in model_frequency.most_common():
        details = model_details.get(model_run, {})
        
        # Calculate average RMSE for this model across groups it appears in
        model_rmse_data = top_models_df[top_models_df['ModelRun'] == model_run]['RMSE']
        avg_rmse = model_rmse_data.mean() if not model_rmse_data.empty else None
        min_rmse = model_rmse_data.min() if not model_rmse_data.empty else None
        max_rmse = model_rmse_data.max() if not model_rmse_data.empty else None
        
        # Get groups where this model appears in top 5
        groups_in_top5 = top_models_df[top_models_df['ModelRun'] == model_run]['Group'].tolist()
        
        frequency_data.append({
            'ModelRun': model_run,
            'Frequency_in_Top5': count,
            'Max_Groups': len(groups),
            'Frequency_Percentage': (count / len(groups)) * 100,
            'Groups_in_Top5': ', '.join(sorted(groups_in_top5)),
            'Avg_RMSE_in_Top5': avg_rmse,
            'Min_RMSE': min_rmse,
            'Max_RMSE': max_rmse,
            'Architecture': details.get('Architecture', 'Unknown'),
            'FeatureNum': details.get('FeatureNum', 'Unknown'),
            'FinalActivation': details.get('FinalActivation', 'Unknown'),
            'Folder': details.get('Folder', 'Unknown')
        })
    
    frequency_df = pd.DataFrame(frequency_data)
    
    # Save frequency analysis
    frequency_csv = f"{outputWorkspace}/model_frequency_in_top_{top_k}.csv"
    frequency_df.to_csv(frequency_csv, index=False)
    print(f"Model frequency analysis saved to: {frequency_csv}")
    
    # Display frequency results
    print(f"\nMOST CONSISTENT MODELS (appear in top {top_k} most often):")
    print("="*70)
    display_cols = ['ModelRun', 'Frequency_in_Top5', 'Frequency_Percentage', 'Avg_RMSE_in_Top5', 'Architecture']
    print(frequency_df[display_cols].head(10).to_string(index=False))
    
    # Highlight the most frequent model
    most_frequent = frequency_df.iloc[0]
    print(f"\n{'='*80}")
    print(f"🏆 MOST CONSISTENT MODEL ACROSS ALL TEST GROUPS:")
    print(f"{'='*80}")
    print(f"Model Run: {most_frequent['ModelRun']}")
    print(f"Appears in top {top_k}: {most_frequent['Frequency_in_Top5']}/{len(groups)} groups ({most_frequent['Frequency_Percentage']:.1f}%)")
    print(f"Groups where it's top {top_k}: {most_frequent['Groups_in_Top5']}")
    print(f"Average RMSE in top {top_k}: {most_frequent['Avg_RMSE_in_Top5']:.6f}")
    print(f"Best RMSE: {most_frequent['Min_RMSE']:.6f}")
    print(f"Worst RMSE: {most_frequent['Max_RMSE']:.6f}")
    print(f"Architecture: {most_frequent['Architecture']}")
    print(f"Features: {most_frequent['FeatureNum']}")
    print(f"Activation: {most_frequent['FinalActivation']}")
    print(f"Folder: {most_frequent['Folder']}")
    
    # Show models that appear in ALL groups
    perfect_models = frequency_df[frequency_df['Frequency_in_Top5'] == len(groups)]
    if not perfect_models.empty:
        print(f"\n🌟 MODELS THAT APPEAR IN TOP {top_k} FOR ALL {len(groups)} GROUPS:")
        print("="*60)
        for _, row in perfect_models.iterrows():
            print(f"• {row['ModelRun']} - {row['Architecture']} (Avg RMSE: {row['Avg_RMSE_in_Top5']:.6f})")
    else:
        print(f"\n⚠️  No models appear in top {top_k} for ALL {len(groups)} groups")
    
    # Show models that appear in most groups
    high_frequency = frequency_df[frequency_df['Frequency_in_Top5'] >= len(groups) * 0.7]  # 70% or more
    if not high_frequency.empty and len(high_frequency) > len(perfect_models):
        print(f"\n⭐ HIGHLY CONSISTENT MODELS (top {top_k} in ≥70% of groups):")
        print("="*60)
        for _, row in high_frequency.iterrows():
            if row['Frequency_in_Top5'] < len(groups):  # Don't repeat perfect models
                print(f"• {row['ModelRun']} - {row['Architecture']} ({row['Frequency_in_Top5']}/{len(groups)} groups, Avg RMSE: {row['Avg_RMSE_in_Top5']:.6f})")
    
    # Display summary
    print(f"\nTOP {top_k} MODELS BY GROUP:")
    print("="*80)
    summary_display = top_models_df[['Overall_Rank', 'Group', 'Rank_in_Group', 'ModelRun', 'RMSE', 'Architecture']].copy()
    print(summary_display.to_string(index=False))
    
    # Show best model per group
    print(f"\nBEST MODEL PER GROUP:")
    print("="*50)
    best_per_group = top_models_df[top_models_df['Rank_in_Group'] == 1][['Group', 'ModelRun', 'RMSE', 'Architecture']]
    print(best_per_group.to_string(index=False))
    
    # Show overall best RMSE
    best_overall = top_models_df.iloc[0]
    print(f"\nBEST OVERALL RMSE:")
    print("="*30)
    print(f"Model Run: {best_overall['ModelRun']}")
    print(f"Group: {best_overall['Group']}")
    print(f"RMSE: {best_overall['RMSE']:.6f}")
    print(f"Architecture: {best_overall['Architecture']}")
    
else:
    print("No valid top models found!")

# Create statistics summary
if len(valid_data) > 0:
    stats_summary = []
    
    # Overall stats
    stats_summary.append({
        'Category': 'Overall',
        'Group': 'All',
        'Count': len(valid_data),
        'Best_RMSE': valid_data['RMSE'].min(),
        'Worst_RMSE': valid_data['RMSE'].max(),
        'Mean_RMSE': valid_data['RMSE'].mean(),
        'Median_RMSE': valid_data['RMSE'].median(),
        'Std_RMSE': valid_data['RMSE'].std()
    })
    
    # Per group stats
    for group in groups:
        group_data = valid_data[valid_data['Group'] == group]
        if not group_data.empty:
            stats_summary.append({
                'Category': 'By_Group',
                'Group': group,
                'Count': len(group_data),
                'Best_RMSE': group_data['RMSE'].min(),
                'Worst_RMSE': group_data['RMSE'].max(),
                'Mean_RMSE': group_data['RMSE'].mean(),
                'Median_RMSE': group_data['RMSE'].median(),
                'Std_RMSE': group_data['RMSE'].std()
            })
    
    # Per architecture stats
    for arch in valid_data['Architecture'].unique():
        if pd.notna(arch) and arch != 'Unknown':
            arch_data = valid_data[valid_data['Architecture'] == arch]
            stats_summary.append({
                'Category': 'By_Architecture',
                'Group': arch,
                'Count': len(arch_data),
                'Best_RMSE': arch_data['RMSE'].min(),
                'Worst_RMSE': arch_data['RMSE'].max(),
                'Mean_RMSE': arch_data['RMSE'].mean(),
                'Median_RMSE': arch_data['RMSE'].median(),
                'Std_RMSE': arch_data['RMSE'].std()
            })
    
    stats_df = pd.DataFrame(stats_summary)
    stats_csv = f"{outputWorkspace}/model_performance_statistics.csv"
    stats_df.to_csv(stats_csv, index=False)
    print(f"\nStatistics summary saved to: {stats_csv}")

print(f"\n{'='*80}")
print("ANALYSIS COMPLETE!")
print(f"{'='*80}")
print(f"Files created:")
print(f"1. {comprehensive_csv} - ALL model data")
print(f"2. {top_models_csv} - Top {top_k} models per group")
print(f"3. {frequency_csv} - Model frequency analysis (MOST IMPORTANT!)")
print(f"4. {stats_csv} - Performance statistics")
print(f"{'='*80}")

modules imported
Loaded architecture stats with 105 records
Processing 23 model runs...
  Processed 10/23 runs...
  Processed 20/23 runs...
Collected data for 133 model/group combinations
Comprehensive results saved to: D:/ASOML/Rockies/modelOutputs/fromAlpine//comprehensive_model_results.csv

Creating top 5 summary...
Found 132 valid model/group combinations
Top 5 models summary saved to: D:/ASOML/Rockies/modelOutputs/fromAlpine//top_5_models_by_group.csv

MODEL FREQUENCY ANALYSIS - TOP 5 ACROSS ALL GROUPS
Model frequency analysis saved to: D:/ASOML/Rockies/modelOutputs/fromAlpine//model_frequency_in_top_5.csv

MOST CONSISTENT MODELS (appear in top 5 most often):
       ModelRun  Frequency_in_Top5  Frequency_Percentage  Avg_RMSE_in_Top5    Architecture
20250721_062531                  4             66.666667          0.124299         FCN_SWE
20250721_063239                  4             66.666667          0.134779         FCN_SWE
20250717_073937                  3             50.0000